In [94]:
import pandas as pd
import numpy as np
from IPython.display import display_html

In [95]:
def get_file_name(state, year) :
    """ returns the filename that contains election results corresponding to a given year and state"""
    
    global_data_path = "C:\\Users\\madhu\\Documents\\git\\TEP\\tep\\data\\raw\\"
    state_year_data_path = global_data_path + str(year) + "\\" + state + "\\"
    if state == "AK" and year >= 2012 :
        return state_year_data_path + "results.txt"
    elif state == "AK" and year < 2012 :
        return state_year_data_path + "results.html"
    else :
        raise ValueError("State " + state + " is not yet supported")

In [90]:
file = get_file_name("AK", 2012)
raw_data = pd.read_csv(file, header = None, encoding = "latin1", sep = " ")
raw_data = raw_data.loc[:,[5,14,17,20,22,23]] # for 2012 only

In [91]:
raw_data.columns = ["race", "data_element", "party_affiliation", "total", "value", "unknown"]

In [93]:
typed_data = pd.DataFrame()
for k in raw_data.columns :
    if raw_data[k].dtype == np.object:
        typed_data[k] = raw_data[k].apply(lambda x: str(x).strip())
    else :
        typed_data[k] = raw_data[k]
dest_file = file.replace("raw","cleaned") + "_cleansed"
typed_data.to_csv(dest_file, index = False)
print(dest_file)
typed_data.head()

C:\Users\madhu\Documents\git\TEP\tep\data\cleaned\2012\AK\results.txt_cleansed


,race,data_element,party_affiliation,total,value,unknown
0,Race Statistics,Number of Precincts,NP,Total,438.0,NaN
1,Race Statistics,Number of Precincts Reporting,NP,Total,438.0,NaN
2,US PRESIDENT,Number of Precincts for Race,NP,Total,438.0,NaN
3,US PRESIDENT,Number of Precincts Reporting,NP,Total,438.0,NaN
4,US PRESIDENT,Registered Voters,NP,Total,506432.0,NaN


In [76]:
typed_data = typed_data.query("data_element in ['Registered Voters','Times Counted']")
typed_data = typed_data[["race","data_element","value"]]
output = pd.pivot_table(typed_data, index = ["race"], values = ["value"], columns = ["data_element"])
print(type(output), output.columns)
output.columns = output.columns.droplevel(0)
output.head()

<class 'pandas.core.frame.DataFrame'> MultiIndex(levels=[['value'], ['Registered Voters', 'Times Counted']],
           codes=[[0, 0], [0, 1]],
           names=[None, 'data_element'])


data_element,Registered Voters,Times Counted
race,,
Ballot Measure 2 - 13PSUM,509011.0,285449.0
Ballot Measure 3 - 13MINW,509011.0,285449.0
Ballot Measure 4 - 12BBAY,509011.0,285449.0
Dis Court JD1 Miller,56357.0,33700.0
Dis Court JD3 Chung,343509.0,194593.0


# Processing HTML files

In [148]:
year = 2000
file = get_file_name("AK", year)
with open(file) as fh:
    data = fh.read()
#display_html(data, raw = True)
raw_data = pd.read_html(data)
len(raw_data)

178

In [150]:
typed_data = pd.DataFrame()
for i in range(3,len(raw_data)):
    if i % 2 == 1:
        df = raw_data[i].loc[2:,:]
        race = raw_data[i].loc[0,0]
        df.insert(loc = 0, column = "race", value = race)
        typed_data = typed_data.append(df, ignore_index = True)
typed_data["race"].unique()

array(['U.S. PRESIDENT/VICE PRESIDENT', 'U.S. REPRESENTATIVE',
       'SENATE DISTRICT A', 'SENATE DISTRICT C', 'SENATE DISTRICT E',
       'SENATE DISTRICT G', 'SENATE DISTRICT I', 'SENATE DISTRICT K',
       'SENATE DISTRICT M', 'SENATE DISTRICT O', 'SENATE DISTRICT Q',
       'SENATE DISTRICT S', 'HOUSE DISTRICT 1', 'HOUSE DISTRICT 2',
       'HOUSE DISTRICT 3', 'HOUSE DISTRICT 4', 'HOUSE DISTRICT 5',
       'HOUSE DISTRICT 6', 'HOUSE DISTRICT 7', 'HOUSE DISTRICT 8',
       'HOUSE DISTRICT 9', 'HOUSE DISTRICT 10', 'HOUSE DISTRICT 11',
       'HOUSE DISTRICT 12', 'HOUSE DISTRICT 13', 'HOUSE DISTRICT 14',
       'HOUSE DISTRICT 15', 'HOUSE DISTRICT 16', 'HOUSE DISTRICT 17',
       'HOUSE DISTRICT 18', 'HOUSE DISTRICT 19', 'HOUSE DISTRICT 20',
       'HOUSE DISTRICT 21', 'HOUSE DISTRICT 22', 'HOUSE DISTRICT 23',
       'HOUSE DISTRICT 24', 'HOUSE DISTRICT 25', 'HOUSE DISTRICT 26',
       'HOUSE DISTRICT 27', 'HOUSE DISTRICT 28', 'HOUSE DISTRICT 29',
       'HOUSE DISTRICT 30', 'HOUSE D

In [151]:
typed_data.columns = ["race", "data_element", "party_affiliation", "value", "unknown"]
dest_file = file.replace("raw","cleaned") + "_cleansed"
typed_data.to_csv(dest_file, index = False)
typed_data.head()

,race,data_element,party_affiliation,value,unknown
0,U.S. PRESIDENT/VICE PRESIDENT,Precincts Reporting,NaN,451/451,100.00%
1,U.S. PRESIDENT/VICE PRESIDENT,Ballots Cast/Reg. Voters,NaN,287825/473648,60.77%
2,U.S. PRESIDENT/VICE PRESIDENT,Total Votes,NaN,285560,NaN
3,U.S. PRESIDENT/VICE PRESIDENT,NaN,NaN,NaN,NaN
4,U.S. PRESIDENT/VICE PRESIDENT,NADER/LaDUKE,GRN,28747,10.07%


In [152]:
typed_data.insert(loc = 0, column = "election_year", value = year)
all_data = all_data.append(typed_data, ignore_index = True) #one time creation of all_data = pd.DataFrame()

In [154]:
all_data.insert(loc = 0, column = "state", value = "AK")
all_data.to_csv(dest_file + "_all", index = False)
all_data.head()

,state,election_year,race,data_element,party_affiliation,value,unknown
0,AK,2010,US REPRESENTATIVE,Number of Precincts,NaN,438,NaN
1,AK,2010,US REPRESENTATIVE,Precincts Reporting,NaN,438,100.0 %
2,AK,2010,US REPRESENTATIVE,Times Counted,NaN,258746/494876,52.3 %
3,AK,2010,US REPRESENTATIVE,Total Votes,NaN,254335,NaN
4,AK,2010,US REPRESENTATIVE,NaN,NaN,NaN,NaN
